In [15]:
def rack_to_pos(rack_str):
    try:
        rack_num = ord(rack_str.upper()) - ord('A') + 1
        rack_num = 6*rack_num - 5
        return rack_num if rack_num >= 1 else 1
    except (AttributeError, TypeError):
      return 0

def rack_to_number(rack_str):
    try:
        rack_num = ord(rack_str.upper()) - ord('A') + 1
        return rack_num
    except (AttributeError, TypeError):
      return

In [16]:
import pandas as pd
import numpy as np

np.random.seed(42)
num_rows = 250
data = {
    'Rack': [chr(np.random.randint(65, 91)) for _ in range(num_rows)],
    'Columna': np.random.randint(1, 53, size=num_rows),
    'Fila': np.random.randint(1, 4, size=num_rows),
    'Cantidad': np.random.randint(1, 100, size=num_rows),
}
df_2D = pd.DataFrame(data)
df_2D

df_2D = df_2D.drop_duplicates(subset=['Rack', 'Columna'])
df_2D['Rack_pos'] = df_2D['Rack'].apply(rack_to_pos)
df_2D = df_2D[df_2D['Rack_pos'] != 0]
df_2D['Fila'] = df_2D['Fila'].astype(int)
df_2D['Rack_pos'] = df_2D['Rack_pos'].astype(int)
df_2D['Cluster'] = [0]*len(df_2D.index)
df_2D = df_2D.sort_values(by='Rack')
df_2D

,Rack,Columna,Fila,Cantidad,Rack_pos,Cluster
190,A,23,3,86,1,0
191,A,15,3,56,1,0
233,A,28,3,90,1,0
26,A,18,1,69,1,0
232,A,19,2,86,1,0
...,...,...,...,...,...,...
79,Z,2,3,65,151,0
99,Z,22,1,47,151,0
124,Z,44,1,79,151,0
28,Z,1,2,25,151,0


In [17]:
from sklearn.cluster import KMeans

def cluster(n):
    X = df_2D[['Rack_pos', 'Columna']]
    
    kmeans = KMeans(n_clusters=n, random_state=0)
    kmeans.fit(X)
    
    labels = kmeans.labels_
    df_2D['Cluster'] = labels

In [18]:
import random
class AntColonyOptimization:
    def __init__(self, distances, n_ants, n_iterations, alpha=1, beta=1, rho=0.5, q=1):
        self.distances = distances
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.alpha = alpha
        self.beta = beta
        self.rho = rho
        self.q = q
        self.pheromone = np.ones_like(distances) / len(distances)

    def run(self):
        best_path = None
        min_distance = float('inf')
        for _ in range(self.n_iterations):
            paths = []
            for _ in range(self.n_ants):
                path, distance = self.find_path()
                paths.append((path, distance))
                if distance < min_distance:
                    min_distance = distance
                    best_path = path
            self.update_pheromone(paths)
        return best_path

    def find_path(self):
        current_node = random.randint(0, len(self.distances) - 1)
        path = [current_node]
        visited = set([current_node])
        distance = 0
        while len(visited) < len(self.distances):
            probabilities = self.calculate_probabilities(current_node, visited)
            next_node = self.choose_next_node(probabilities)
            path.append(next_node)
            visited.add(next_node)
            distance += self.distances[current_node][next_node]
            current_node = next_node
        return path, distance

    def calculate_probabilities(self, current_node, visited):
        probabilities = []
        for next_node in range(len(self.distances)):
            if next_node not in visited:
                pheromone = self.pheromone[current_node][next_node]
                distance = self.distances[current_node][next_node]
                probability = (pheromone ** self.alpha) * ((1 / distance) ** self.beta)
                probabilities.append((next_node, probability))
        return probabilities

    def choose_next_node(self, probabilities):
        total_prob = sum([prob for _, prob in probabilities])
        random_value = random.uniform(0, total_prob)
        current_prob = 0
        for node, prob in probabilities:
            current_prob += prob
            if current_prob >= random_value:
                return node
        return probabilities[-1][0] 

    def update_pheromone(self, paths):
        self.pheromone *= (1 - self.rho)
        for path, distance in paths:
            for i in range(len(path) - 1):
                self.pheromone[path[i]][path[i+1]] += self.q / distance

In [42]:
def get_best_route(n):
    cluster_n_df = df_2D[df_2D['Cluster'] == n]
    coordinates = cluster_n_df[['Rack_pos', 'Columna']].values

    distances = np.zeros((len(coordinates), len(coordinates)))
    for i in range(len(coordinates)):
        for j in range(i + 1, len(coordinates)):
            distance = np.linalg.norm(coordinates[i] - coordinates[j])
            distances[i, j] = distances[j, i] = distance

    aco = AntColonyOptimization(distances, n_ants=10, n_iterations=100)
    best_path = aco.run()

    route = cluster_n_df.iloc[best_path][['Rack', 'Columna']].values

    data = [{'Index': index, 'Rack': item[0], 'Columna': item[1]} for index, item in enumerate(route)]

    return data

In [1]:
from flask import Flask, jsonify, request

app = Flask(__name__)

@app.route('/get_data', methods=['GET'])
def get_dataframe():
    return jsonify(df_2D.to_dict(orient='records'))


@app.route('/get_clustered_data', methods=['POST'])
def post_get_clustered_dataframe():
    data = request.get_json()
    n = data.get('n')
    n = int(n)
    cluster(n)
    return jsonify(df_2D.to_dict(orient='records'))

@app.route('/get_best_route', methods=['POST'])
def post_tuples():
    data = request.get_json()
    n = data.get('n')
    n = int(n)
    res = get_best_route(n)
    return jsonify(res)

if __name__ == '__main__':
    app.run(debug=False, port=8080)


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8080 is in use by another program. Either identify and stop that program, or start the server with a different port.


SystemExit: 1

/opt/anaconda3/envs/ProyectoBimbo/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


import sys
import pandas as pd
from PyQt5.QtWidgets import QApplication, QMainWindow, QVBoxLayout, QPushButton, QWidget, QSpinBox, QLabel, QHBoxLayout
from PyQt5.QtWebEngineWidgets import QWebEngineView
import plotly.graph_objects as go
from sklearn.cluster import KMeans

class MainWindow(QMainWindow):
    def __init__(self, df_2D):
        super().__init__()
        self.df_2D = df_2D

        self.setWindowTitle("Bimbot rutas")
        self.setGeometry(100, 100, 800, 600)

        layout = QVBoxLayout()

        self.plot_button = QPushButton("Mostrar Estado del Almacén")
        self.plot_button.clicked.connect(lambda: self.show_plot())
        layout.addWidget(self.plot_button)

        cluster_layout = QHBoxLayout()
        cluster_label = QLabel("Seleccione numero de grupos:")
        cluster_layout.addWidget(cluster_label)

        self.n_spinbox = QSpinBox()
        self.n_spinbox.setMinimum(1)
        self.n_spinbox.setMaximum(len(df_2D.index))
        cluster_layout.addWidget(self.n_spinbox)

        self.cluster_button = QPushButton("Agrupar")
        self.cluster_button.clicked.connect(lambda: self.show_clustered_plot(self.n_spinbox.value()))
        cluster_layout.addWidget(self.cluster_button)

        layout.addLayout(cluster_layout)

        route_layout = QHBoxLayout()
        route_label = QLabel("Seleccione grupo a calcular ruta")
        route_layout.addWidget(route_label)

        self.n_route_spinbox = QSpinBox()
        self.n_route_spinbox.setMinimum(1)
        self.n_route_spinbox.setMaximum(1000)
        route_layout.addWidget(self.n_route_spinbox)

        self.route_button = QPushButton("Calcular ruta optima")
        self.route_button.clicked.connect(lambda: self.show_route_plot(self.n_route_spinbox.value()))
        route_layout.addWidget(self.route_button)

        layout.addLayout(route_layout)

        self.graph_view = QWebEngineView()
        layout.addWidget(self.graph_view)

        container = QWidget()
        container.setLayout(layout)
        self.setCentralWidget(container)

        self.setStyleSheet(open('../Utils/style.css').read())

    def show_plot(self):
        fig = go.Figure(data=go.Scattergl(
            x=self.df_2D['Rack'],
            y=self.df_2D['Columna'],
            mode='markers',
            marker=dict(
                size=12,
                color=self.df_2D['Cantidad'],
                colorscale='Viridis',
                showscale=True
            ),
            text=self.df_2D['Cantidad'],
            hoverinfo='text+x+y'
        ))

        fig.update_layout(
            title='Cantidad por Rack y Columna',
            xaxis_title='Rack',
            yaxis_title='Columna'
        )

        html = fig.to_html(include_plotlyjs='cdn')
        self.graph_view.setHtml(html)


    def create_clustered_plot(self, n):
        X = df_2D[['Rack_pos', 'Columna']]

        kmeans = KMeans(n_clusters=n, random_state=0)
        kmeans.fit(X)

        labels = kmeans.labels_
        df_2D['Cluster'] = labels

        fig = go.Figure()

        unique_clusters = df_2D['Cluster'].unique()
        for cluster in unique_clusters:
            cluster_data = df_2D[df_2D['Cluster'] == cluster]
            fig.add_trace(go.Scatter(
                x=cluster_data['Rack'],
                y=cluster_data['Columna'],
                mode='markers',
                marker=dict(size=12, color=cluster),
                name=f'Grupo {cluster}'
            ))

        fig.update_layout(title='Grupo',
                          xaxis_title='Rack',
                          yaxis_title='Columna')

        return fig

    def show_clustered_plot(self, n):
        clustered_fig = self.create_clustered_plot(n)
        html = clustered_fig.to_html(include_plotlyjs='cdn')
        self.graph_view.setHtml(html)


    def create_route_plot(self, n):
        cluster_n_df = df_2D[df_2D['Cluster'] == n]
        coordinates = cluster_n_df[['Rack_pos', 'Columna']].values

        distances = np.zeros((len(coordinates), len(coordinates)))
        for i in range(len(coordinates)):
            for j in range(i + 1, len(coordinates)):
                distance = np.linalg.norm(coordinates[i] - coordinates[j])
                distances[i, j] = distances[j, i] = distance

        aco = AntColonyOptimization(distances, n_ants=10, n_iterations=100)
        best_path, min_distance = aco.run()

        import plotly.graph_objects as go

        best_path_coordinates = cluster_n_df.iloc[best_path][['Rack', 'Columna']].values

        fig = go.Figure()

        unique_clusters = df_2D['Cluster'].unique()
        for cluster in unique_clusters:
            cluster_data = df_2D[df_2D['Cluster'] == cluster]
            fig.add_trace(go.Scatter(
                x=cluster_data['Rack'],
                y=cluster_data['Columna'],
                mode='markers',
                marker=dict(size=8, color=cluster),
                name=f'Grupo {cluster}'
            ))


        fig.add_trace(go.Scatter(
            x=best_path_coordinates[:, 0],
            y=best_path_coordinates[:, 1],
            mode='lines+markers',
            marker=dict(size=10, color='red'),
            line=dict(width=2, color='red'),
            name='Ruta Optima'
        ))

        fig.update_layout(title=f'Ruta optima del grupo {n}',
                          xaxis_title='Rack',
                          yaxis_title='Columna')

        return fig


    def show_route_plot(self, n):
        route_fig = self.create_route_plot(n)
        html = route_fig.to_html(include_plotlyjs='cdn')
        self.graph_view.setHtml(html)


def main(df_2D):
    app = QApplication(sys.argv)
    window = MainWindow(df_2D)
    window.show()
    app.exec_()


if __name__ == "__main__":
    main(df_2D)